In [1]:
from ai import ai_abs_dls, ai14s
from marubatsu import Marubatsu

mb = Marubatsu()
eval_params = {"minimax": True}

In [2]:
%%timeit

ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=9)

68.1 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from ai import show_progress, ai2, ai11s

show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

TypeError: cannot unpack non-iterable NoneType object

In [4]:
def show_progress(ai, winner, params=[{}, {}]):
    mb = Marubatsu()
    while True:
        if mb.play(ai=ai, verbose=False, params=params) == winner:
            records = mb.records
            mb.restart()
            for x, y in records[1:]:
                mb.move(x, y)
                print(mb)
            break

In [5]:
show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

Turn x
...
...
O..

Turn o
...
.X.
o..

Turn x
..O
.x.
o..

Turn o
X.o
.x.
o..

Turn x
x.o
.x.
o.O

Turn o
xXo
.x.
o.o

winner o
xxo
.x.
oOo



In [6]:
mb = Marubatsu()
print(mb.records)

[None]


In [7]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [8]:
mb = Marubatsu()
print(mb.records)

[(-1, -1)]


In [9]:
def remove_mark(self, x, y):
    if 0 <= x < self.BOARD_SIZE and 0 <= y < self.BOARD_SIZE:
        if self.board[x][y] != Marubatsu.EMPTY:
            self.board[x][y] = Marubatsu.EMPTY
            return True
        else:
            print("(", x, ",", y, ") のマスにはマークがありません")
            return False
    else:
        print("(", x, ",", y, ") はゲーム盤の範囲外の座標です")
        return False   
    
Marubatsu.remove_mark = remove_mark

In [10]:
mb = Marubatsu()
mb.move(1, 1)
print(mb)
mb.remove_mark(1, 1)
print(mb)

Turn x
...
.O.
...

Turn x
...
...
...



In [11]:
a = [1, 2, 3]
b = a.pop()
print(a)
print(b)

[1, 2]
3


In [12]:
a = [1, 2, 3]
b = a.pop(1)
print(a)
print(b)

[1, 3]
2


In [13]:
a = {
    "x": 1,
    "y": 2,
    "z": 3,
}
b = a.pop("z")
print(a)
print(b)

{'x': 1, 'y': 2}
3


In [14]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]
            
Marubatsu.unmove = unmove

In [15]:
mb = Marubatsu()
mb.move(0, 0)
mb.move(1, 0)
mb.move(2, 0)
mb.move(0, 1)
mb.move(1, 1)
for _ in range(5):
    mb.unmove()
    print(mb)

Turn o
oxo
X..
...

Turn x
oxO
...
...

Turn o
oX.
...
...

Turn x
O..
...
...

Turn o
...
...
...



In [16]:
from ai import ai_by_mmscore, dprint
from time import perf_counter

@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, use_tt=False, tt=None, tt_for_mo=None):           
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if not use_tt:            
                boardtxt = mborig.board_to_str()
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]        
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig, bestmove)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt, move in boardtxtlist.items():
                tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [17]:
%%timeit

ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=9)

36.8 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
from util import Check_solved

params = {
    "eval_func": ai14s,
    "eval_params": eval_params,
    "use_tt": True,
    "maxdepth": 9
}

Check_solved.is_strongly_solved(ai_abs_dls, params=params)

100%|██████████| 431/431 [00:01<00:00, 308.63it/s]

431/431 100.00%


(True, [])